# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7e205b1fa0>
├── 'a' --> tensor([[ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433]])
└── 'x' --> <FastTreeValue 0x7f7e205b19d0>
    └── 'c' --> tensor([[ 0.0573,  0.2263,  0.5933, -0.8148],
                        [ 0.6951,  0.2651, -0.6542,  0.8168],
                        [-0.9352, -0.0705,  1.5325,  0.8825]])

In [4]:
t.a

tensor([[ 1.3848, -0.5953, -1.7745],
        [ 0.0748, -0.7428, -0.2433]])

In [5]:
%timeit t.a

86.9 ns ± 1.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7e205b1fa0>
├── 'a' --> tensor([[ 0.3227,  0.0451, -0.5650],
│                   [-0.6428,  0.7951,  1.0529]])
└── 'x' --> <FastTreeValue 0x7f7e205b19d0>
    └── 'c' --> tensor([[ 0.0573,  0.2263,  0.5933, -0.8148],
                        [ 0.6951,  0.2651, -0.6542,  0.8168],
                        [-0.9352, -0.0705,  1.5325,  0.8825]])

In [7]:
%timeit t.a = new_value

128 ns ± 1.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3848, -0.5953, -1.7745],
               [ 0.0748, -0.7428, -0.2433]]),
    x: Batch(
           c: tensor([[ 0.0573,  0.2263,  0.5933, -0.8148],
                      [ 0.6951,  0.2651, -0.6542,  0.8168],
                      [-0.9352, -0.0705,  1.5325,  0.8825]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3848, -0.5953, -1.7745],
        [ 0.0748, -0.7428, -0.2433]])

In [11]:
%timeit b.a

69.6 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1950, -0.5695, -1.2050],
               [-0.6962,  1.3274, -1.0827]]),
    x: Batch(
           c: tensor([[ 0.0573,  0.2263,  0.5933, -0.8148],
                      [ 0.6951,  0.2651, -0.6542,  0.8168],
                      [-0.9352, -0.0705,  1.5325,  0.8825]]),
       ),
)

In [13]:
%timeit b.a = new_value

611 ns ± 17.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 21.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.5 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

324 µs ± 36.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

287 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7e205a3c70>
├── 'a' --> tensor([[[ 1.3848, -0.5953, -1.7745],
│                    [ 0.0748, -0.7428, -0.2433]],
│           
│                   [[ 1.3848, -0.5953, -1.7745],
│                    [ 0.0748, -0.7428, -0.2433]],
│           
│                   [[ 1.3848, -0.5953, -1.7745],
│                    [ 0.0748, -0.7428, -0.2433]],
│           
│                   [[ 1.3848, -0.5953, -1.7745],
│                    [ 0.0748, -0.7428, -0.2433]],
│           
│                   [[ 1.3848, -0.5953, -1.7745],
│                    [ 0.0748, -0.7428, -0.2433]],
│           
│                   [[ 1.3848, -0.5953, -1.7745],
│                    [ 0.0748, -0.7428, -0.2433]],
│           
│                   [[ 1.3848, -0.5953, -1.7745],
│                    [ 0.0748, -0.7428, -0.2433]],
│           
│                   [[ 1.3848, -0.5953, -1.7745],
│                    [ 0.0748, -0.7428, -0.2433]]])
└── 'x' --> <FastTreeValue 0x7f7e205a3a90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

49.8 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7d48a1b490>
├── 'a' --> tensor([[ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433],
│                   [ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433],
│                   [ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433],
│                   [ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433],
│                   [ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433],
│                   [ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433],
│                   [ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433],
│                   [ 1.3848, -0.5953, -1.7745],
│                   [ 0.0748, -0.7428, -0.2433]])
└── 'x' --> <FastTreeValue 0x7f7d48b84910>
    └── 'c' --> tensor([[ 0.0573,  0.2263,  0.5933, -0.8148],
                        [ 0.6951,  0.2651, -0.6542,  0.8168],
                 

In [23]:
%timeit t_cat(trees)

46.1 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

101 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0573,  0.2263,  0.5933, -0.8148],
                       [ 0.6951,  0.2651, -0.6542,  0.8168],
                       [-0.9352, -0.0705,  1.5325,  0.8825]],
              
                      [[ 0.0573,  0.2263,  0.5933, -0.8148],
                       [ 0.6951,  0.2651, -0.6542,  0.8168],
                       [-0.9352, -0.0705,  1.5325,  0.8825]],
              
                      [[ 0.0573,  0.2263,  0.5933, -0.8148],
                       [ 0.6951,  0.2651, -0.6542,  0.8168],
                       [-0.9352, -0.0705,  1.5325,  0.8825]],
              
                      [[ 0.0573,  0.2263,  0.5933, -0.8148],
                       [ 0.6951,  0.2651, -0.6542,  0.8168],
                       [-0.9352, -0.0705,  1.5325,  0.8825]],
              
                      [[ 0.0573,  0.2263,  0.5933, -0.8148],
                       [ 0.6951,  0.2651, -0.6542,  0.8168],
                       [-0.9352, -0.0705,  1.5325,  0.8825]],

In [26]:
%timeit Batch.stack(batches)

109 µs ± 3.81 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0573,  0.2263,  0.5933, -0.8148],
                      [ 0.6951,  0.2651, -0.6542,  0.8168],
                      [-0.9352, -0.0705,  1.5325,  0.8825],
                      [ 0.0573,  0.2263,  0.5933, -0.8148],
                      [ 0.6951,  0.2651, -0.6542,  0.8168],
                      [-0.9352, -0.0705,  1.5325,  0.8825],
                      [ 0.0573,  0.2263,  0.5933, -0.8148],
                      [ 0.6951,  0.2651, -0.6542,  0.8168],
                      [-0.9352, -0.0705,  1.5325,  0.8825],
                      [ 0.0573,  0.2263,  0.5933, -0.8148],
                      [ 0.6951,  0.2651, -0.6542,  0.8168],
                      [-0.9352, -0.0705,  1.5325,  0.8825],
                      [ 0.0573,  0.2263,  0.5933, -0.8148],
                      [ 0.6951,  0.2651, -0.6542,  0.8168],
                      [-0.9352, -0.0705,  1.5325,  0.8825],
                      [ 0.0573,  0.2263,  0.5933, -0.8148],
                   

In [28]:
%timeit Batch.cat(batches)

193 µs ± 4.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

723 µs ± 26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
